# Graph Coloring

The **"Graph Coloring"** quantum kata is a series of exercises designed
to teach you the basics of using Grover search to solve constraint
satisfaction problems, using graph coloring problem as an example.

* You can read more about graph coloring problems [here](https://en.wikipedia.org/wiki/Graph_coloring).
* It is strongly recommended to complete the [Grover's Algorithm kata](./../GroversAlgorithm/GroversAlgorithm.ipynb) before proceeding to this one. You can also refer to its [README.md](./../GroversAlgorithm/README.md) for the list of resources on Grover's algorithm.
* [SolveSATWithGrover](./../SolveSATWithGrover/SolveSATWithGrover.ipynb) is another kata covering oracle implementation for solving constraint satisfaction problems.


Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blank (marked with the `// ...` comments)
with some Q# code that solves the task. To verify your answer, run the cell using Ctrl+Enter (⌘+Enter on macOS).

Within each section, tasks are given in approximate order of increasing difficulty; 
harder ones are marked with asterisks.

## Part I. Colors Representation and Manipulation

### Task 1.1. Initialize register to a color

**Inputs:** 

  1. An integer $C$ ($0 \leq C \leq 2^{N} - 1$).

  2. An array of $N$ qubits in the $|0...0\rangle$ state.

**Goal:** 

Prepare the array in the basis state which represents the binary notation of $C$. 
Use little-endian encoding (i.e., the least significant bit should be stored in the first qubit).

**Example:** For $N = 2$ and $C = 2$ the state should be $|01\rangle$.

### Solution

We first need to convert C from an integer to a binary representation. In Q#, we can use the [IntAsBoolArray](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.convert.intasboolarray) function to convert the input index its equivalent binary representation `binaryC`.

Next we need to use `binaryC` as a BitMask and whenever `binaryC[i]` is `One` we need to flip the qubit by applying `PauliX` gate. Although we can do this manually via a for loop, Q# has a helpful operation called [ApplyPauliFromBitString](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.canon.applypaulifrombitstring). 
`ApplyPauliFromBitString` takes input a Pauli Matrix, a boolean array and a qubit register and applies the Pauli Matrix according to the BitString.

We can think of `ApplyPauliFromBitString` implementing the Unitary 

$$P^{b_0} \otimes P^{b_1} \otimes ... \otimes P^{b_{n-1}}$$

where $P^0=I, P^1 =P$ and $P \in \{I,X,Y,Z\}$ is a Pauli Matrix and $b_i \in \{0,1\}$ such that $b_0b_1...b_{n-1}$ representing the BitString/Boolean Array. 
`ApplyPauliFromBitString(PauliX,true,binaryC,register)` represents the transformation of `register`

$$|\psi\rangle \xrightarrow{} X^{c_0} \otimes X^{c_1} \otimes ... \otimes X^{c_{n-1}}|\psi\rangle$$

where $c_0c_1...c_{n-1}$ is the binary representation of $C$.

When the input qubit register is in the state $|0...0\rangle$, the `ApplyPauliFromBitString` operation will convert it into a basis state representing the little-endian encoding of the boolean array i.e. little-endian binary encoding of $C$

$$|0...0\rangle \xrightarrow{} X^{c_0} \otimes X^{c_1} \otimes ... \otimes X^{c_{n-1}}|0...0\rangle = |c_0c_1...c_{n-1}\rangle$$


In [ ]:
%kata T11_InitializeColor_Test 

open Microsoft.Quantum.Convert;
operation InitializeColor (C : Int, register : Qubit[]) : Unit is Adj {
    let N = Length(register);
    // Convert C to an array of bits in little endian format
    let binaryC = IntAsBoolArray(C, N);
    // Value "true" corresponds to bit 1 and requires applying an X gate
    ApplyPauliFromBitString(PauliX, true, binaryC, register);
}

[Return to task 1.1 of the Graph Coloring kata.](./GraphColoring.ipynb#Task-1.1.-Initialize-register-to-a-color)

### Task 1.2. Read color from a register

**Input:** An array of $N$ qubits which are guaranteed to be in one of the $2^{N}$ basis states.

**Output:** 

An $N$-bit integer that represents this basis state, in little-endian encoding. 
The operation should not change the state of the qubits.

**Example:** For $N = 2$ and the qubits in the state $|01\rangle$ return 2 (and keep the qubits in $|01\rangle$).

### Solution

Since we are guaranteed that `register` is in one of the $2^N$ basis states. 
Simply Measuring it (without `Reset`), will not destroy any superposition and also leave the state of the qubits unchanged. 
We can use the [MultiM](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.measurement.multim) Operation to measure all the qubits together and store the result in `measurements` which is of the type `Result[]`. 

We now need to convert these bits into an integer. We can do this directly by using the [ResultArrayAsInt](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.convert.resultarrayasint) function which convert an array of `Result` representing a bitstring(or a binary number) into the equivalent integer $N$.

In [ ]:
%kata T12_MeasureColor_Test 

open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Measurement;
operation MeasureColor (register : Qubit[]) : Int {
    let measurements = MultiM(register);
    return ResultArrayAsInt(measurements);
}

[Return to task 1.2 of the Graph Coloring kata.](./GraphColoring.ipynb#Task-1.2.-Read-color-from-a-register)

### Task 1.3. Read coloring from a register

**Inputs:** 

  1. The number of elements in the coloring $K$.

  2. An array of $K * N$ qubits which are guaranteed to be in one of the $2^{KN}$ basis states.

**Output:** 

An array of $K$ $N$-bit integers that represent this basis state. 
$i$-th integer of the array is stored in qubits with indices $i * N$, $i * N + 1$, ..., $i * N + N - 1$ in little-endian format. 
The operation should not change the state of the qubits.

**Example:** 
For $N = 2$, $K = 2$ and the qubits in the state $|0110\rangle$ return `[2, 1]`.

### Solution

This can be considered as $K$-register version of Task 2.2. In Task 2.2 We read the color from a register of size $N$.
Here we have to return $K$ integers representing the $K$ colors from each of the $N$ bit sized registers. 

We are given $K$ and hence we can find out $N$ by measuring the `Length` of the register and dividing it by $K$. 

Next we need to divide $KN$-qubit sized register into $K$ $N$-qubit sized registers. 
We use the Q# function [Partitioned](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.arrays.partitioned) which takes input an array of integers representing the number of elements in each partition and the array to be partitioned.

`colorPartitions = Partitioned(ConstantArray(K-1,N),register)`where `ConstantArray(K-1,N)` generates an array of size $K-1$ with each element being $N$. `colorPartitions` is thus an array of size $K$ consisting of the $N$-qubit sized registers. 

>Note: The Length of array is $K-1$ since Partitioned adds the remaining elements not inserted in prior partitions into the last partition. 

We use the `ForEach` operation to apply the `MeasureColor` operation from Task 2.2 to all elements of `colorPartitions` and return the resulting array `coloring`.

In [ ]:
%kata T13_MeasureColoring_Test 

open Microsoft.Quantum.Arrays;
operation MeasureColoring (K : Int, register : Qubit[]) : Int[] {
    let N = Length(register) / K;
    let colorPartitions = Partitioned(ConstantArray(K - 1, N), register);
    let coloring = ForEach(MeasureColor, colorPartitions);
    return coloring;
}

[Return to task 1.3 of the Graph Coloring kata.](./GraphColoring.ipynb#Task-1.3.--Read-coloring-from-a-register)

### Task 1.4. 2-bit color equality oracle

**Inputs:** 

  1. An array of 2 qubits in an arbitrary state $|c_{0}\rangle$ representing the first color.

  2. An array of 2 qubits in an arbitrary state $|c_{1}\rangle$ representing the second color.

  3. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:**

Transform state $|c_{0}\rangle|c_{1}\rangle|y\rangle$ into state $|c_{0}\rangle|c_{1}\rangle|y \oplus f(c_{0},c_{1})\rangle$ ($\oplus$ is addition modulo 2), 
where $f(x) = 1$ if $c_{0}$ and $c_{1}$ are in the same state, and 0 otherwise. 
Leave the query register in the same state it started in.

In this task you are allowed to allocate extra qubits.

### Solution

We are given that $f(c_0,c_1)=1$ if and only if $c_0=c_1$. $XOR$ or $\oplus$ is a binary operation which satisifies the condition that 
   
$$c_0 = c_1 \implies c_0 \oplus c_1 =0 $$
We can thus calculate $f(c_0,c_1)$ as $1$ if $c_0\oplus c_1=0$ and $0$ otherwise. The Advantage of this is that we can calculate the $XOR$ of $c_0,c_1$ using the $CNOT$ operator. 

We allocate two ancillary qubits $|a\rangle$ which are in the initial state $|00\rangle$. 
We can then compute $XOR$ (and later uncompute) in this extra register by using 4 $CNOT$ operations.
The $XOR$ can be computed in the following manner 
```
CNOT(c0[0],a[0]);
CNOT(c0[1],a[1]);
CNOT(c1[0],a[0]);
CNOT(c1[1],a[1]);
```
The state of the $|a\rangle$ after this operation would be $|c_0 \oplus c_1\rangle$

We now need to flip the target qubit $|y\rangle$ only if $|a_0a_1\rangle = |00\rangle$. 
This can be done by using $0$ Controlled $X$ gate i.e. `ControlledOnInt(0,X)`
This ensures that $|y\rangle$ is transformed into $|y \oplus f(c_0,c_1)\rangle$. 
Finally we also need to uncompute the $XOR$ to ensure the ancillary qubits are again in the $|00\rangle$ state.

In [ ]:
%kata T14_ColorEqualityOracle_2bit_Test 

operation ColorEqualityOracle_2bit (c0 : Qubit[], c1 : Qubit[], target : Qubit) : Unit is Adj+Ctl {
    using(a = Qubit[2]){
        within {
            // compute XOR of c0 and c1 and storing it in a0a1
            CNOT(c0[0],a[0]);
            CNOT(c0[1],a[1]);
            CNOT(c1[0],a[0]);
            CNOT(c1[1],a[1]);
        } apply {
            // if all XORs are 0, c0 = c1
            (ControlledOnInt(0, X))(a, target);
        }
    }
}

[Return to task 1.4 of the Graph Coloring kata.](./GraphColoring.ipynb#Task-1.4.-2-bit-color-equality-oracle)

### Task 1.5. N-bit color equality oracle (no extra qubits)

This task is the same as task 1.4, but in this task you are NOT allowed to allocate extra qubits.

### Solution

Since this Task is the generalised $N$ bit version of the previous task, we need to compute the $N$-bit sized $XOR$ of $c_0$ and $c_1$. 
However we are not allowed to allocate extra qubits and thus must compute (and uncompute) it in-place.

This can be done by storing $c_1 \oplus c_0$ in $c_1$ itself.
We need to use $N$ CNOT gates with the qubits of $c_0$ serving as control and the respective qubits of $c_1$ acting as the target.
The remaining procedure is exactly the same as Task 1.4

In [ ]:
%kata T15_ColorEqualityOracle_Nbit_Test 

operation ColorEqualityOracle_Nbit (c0 : Qubit[], c1 : Qubit[], target : Qubit) : Unit is Adj+Ctl {
    within {
        // compute XOR of q0 and q1 in place (storing it in q1)
        for (i in 0..Length(c0)-1) {
            CNOT(c0[i], c1[i]);
        }
    } apply {
        // if all XORs are 0, the bit strings are equal
        (ControlledOnInt(0, X))(c1, target);
    }
}

[Return to task 1.5 of the Graph Coloring kata.](./GraphColoring.ipynb#Task-1.5.-N-bit-color-equality-oracle-(no-extra-qubits))

## Part II. Vertex coloring problem

### Task 2.1. Classical verification of vertex coloring

**Inputs:** 

  1. The number of vertices in the graph $V$ ($V \leq 6$).

  2. An array of $E$ tuples of integers, representing the edges of the graph ($E \leq 12$).  
Each tuple gives the indices of the start and the end vertices of the edge.  
The vertices are indexed $0$ through $V - 1$.

  3. An array of $V$ integers, representing the vertex coloring of the graph. 
$i$-th element of the array is the color of the vertex number $i$.

**Output:** 

True if the given vertex coloring is valid (i.e., no pair of vertices connected by an edge have the same color), and false otherwise.

**Example:** 

Graph 0 -- 1 -- 2 would have $V = 3$ and `edges = [(0, 1), (1, 2)]`.  
Some of the valid colorings for it would be `[0, 1, 0]` and `[-1, 5, 18]`.

### Solution

A graph is colored correctly when the nodes of every edge have a different color. This means that we have to check every edge, see if the nodes have the same color and if it is the case return that the graph is incorrect. If every edge passed the test, we can safely say the graph is correctly coloured.

Since the color of vertex $n$  is element $n$ of the color array, we simply loop through every edge, which is a pair of vertices.

In [ ]:
%kata T21_IsVertexColoringValid_Test 

function IsVertexColoringValid (V : Int, edges: (Int, Int)[], colors: Int[]) : Bool {
    // We loop through every element
    for ((v0, v1) in edges){
        // We compare the colors
        if (colors[v0] == colors[v1]){
            //A return statement stops the execution of the function$
            return false;
        }
    }
    // If the code reaches this point, every edge was correct
    return true;
}

[Return to task 2.1 of the Graph Coloring kata.](./GraphColoring.ipynb#Task-2.1.-Classical-verification-of-vertex-coloring)

### Task 2.2. Oracle for verifying vertex coloring

**Inputs:**

  1. The number of vertices in the graph $V$ ($V \leq 6$).

  2. An array of $E$ tuples of integers, representing the edges of the graph (E $\leq$ 12).  
Each tuple gives the indices of the start and the end vertices of the edge.  
The vertices are indexed $0$ through $V - 1$.

  3. An array of $2V$ qubits `colorsRegister` that encodes the color assignments.

  4. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:**

Transform state $|x, y\rangle$ into state $|x, y \oplus f(x)\rangle$  ($\oplus$ is addition modulo 2), 
where $f(x) = 1$ if if the given vertex coloring is valid, and 0 otherwise. 
Leave the query register in the same state it started in.

Each color in `colorsRegister` is represented as a 2-bit integer in little-endian format. 
See task 1.3 for a more detailed description of color assignments.

### Solution

The oracle will work the same way the ColorEqualityOracle_Nbit works. We will have an array of qubits, of the same size as the number of edges, that will be flipped if the vertices of the edge have the same color. This can easily be done with the operation defined in task 1.5. We will then check if the array is still in state $|0...0\rangle$.

However, as the coloring is provided as an array of qubits, with 2 qubits per vertex (2 qubits = 4 basis states = 4 colors), we have to take the correct chunks of the coloring. We can deduce that the coloring of vertex $n$ is from position $2*n$ to position $2*n+1$.

Also do not forget to uncompute to leave the qubits clean.

Note 1 : do not forget that element numeber $a$ to element number $b$ of an array is written as `array[a..b]` ([docs](https://docs.microsoft.com/en-us/quantum/user-guide/language/types#array-types))  
Note 2 : applying code $a$, then code $b$, then code $a$ again, can be done with the `within {} apply {}` structure ([docs](https://docs.microsoft.com/en-us/quantum/user-guide/using-qsharp/control-flow#conjugations))

In [ ]:
%kata T22_VertexColoringOracle_Test 

operation VertexColoringOracle (V : Int, 
                                edges : (Int, Int)[], 
                                colorsRegister : Qubit[], 
                                target : Qubit) : Unit is Adj+Ctl {
    // We store the number of edges
    let numberEdges = Lenght(edges);
    // We allocate the qubits to the array storing the correctness of the coloring 
    using (conflicts = Qubit(numberEdges)) {
        within {
            // We iterate over every edge
            for (i in 0..numberEdges-1) {
                // We store the edge in two separate variables, one for each vertex
                let (v0, v1) = edges[i];
                // We use the operation from task 1.5
                ColorEqualityOracle_Nbit(colorsRegister[2*v0..2*v0+1], colorsRegister[2*v1..2*v1+1], conflicts[i]);
            }
        } apply {
            // If all the tests passed, conflicts should be in state |0...0⟩
            (ControlledOnInt(0, X))(correctness, target);
        }
    }
}

[Return to task 2.2 of the Graph Coloring kata.](./GraphColoring.ipynb#Task-2.2.-Oracle-for-verifying-vertex-coloring)

### Task 2.3. Using Grover's search to find vertex coloring

**Inputs:** 

  1. The number of vertices in the graph $V$ ($V \leq 6$).

  2. A marking oracle which implements vertex coloring verification, as implemented in task 2.2.

**Output:** 

A valid vertex coloring for the graph, in a format used in task 2.1.

### Solution

First we have to implement Grover's Algorithm. However, this is not the goal of this kata so we will simply assume that the Grover's Algorithm kata was already completed and understood. The first block of code is the phase kickback trick to turn the marking oracle into a phase flip algorithm. The second is the actual implementation of Grover's Algorithm.

The main part requires us to run Grover's Algorithm to find a correct solution. Then, we have to measure the solution and see if the measurement is correct with the oracle. Finnally, if the solution is correct, measure the solution using the MeasureColoring operation from part 1.3.

The real implementation of the graph coloring problem will work as follows
- Create an array of integers which will store the coloring
- Borrow an array with 2 qubits per vertex and borrow one more qubit as a verification qubit
- Create a number to store the number of iterations Grover's Algorithm should be using
- Create a bolean which will store if we found a correct solution, set it to false
- Repeat until it works or the number of iterations is over 10 :
    - Use Grover's Algorithm with the array of qubits, the oracle and the number of iterations
    - Measure the result of Grover's Algorithm
    - Verify the solution using the oracle, use the verification qubit
    - Measure the verification qubit in the Pauli Z basis and reset it
        - If the solution is correct, set coloring to true and use the MeasureColoring operation from part 1.3
        - If the solution is incorrect, do nothing
    - Reset the coloring register
- If a solution was found, return it
- If no solution was found, return nothing

Note 1 : The `repeat {} until () fixup {}` control flow can be used for this ([docs](https://docs.microsoft.com/en-us/quantum/user-guide/using-qsharp/control-flow#repeat-until-success-loop))  
Note 2 : The measurement of an array can be done using the `MultiM` operation, this will measure the array in the PauliZ ([docs](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.measurement.multim))  
Note 3 : Measuring a qubit in the PauliZ basis and measuring can be done using `MResetZ` ([docs](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.measurement.mresetz))  
Note 4 : Reseting a register can be done using the `ResetAll` operation ([docs](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.resetall))

In [ ]:
operation oracleConverter (markingOracle : ((Qubit[], Qubit) => Unit is Adj), register : Qubit[]) : Unit is Adj {
    using (target = Qubit()) {
        within {
            // We put the target qubit in the |-⟩ state
            X(target);
            H(target);
        } apply {
            // We apply the marking oracle
            markingOracle(register, target);
        }
    }
}

In [ ]:
open Microsoft.Quantum.Arrays;

operation groverAlgorithm (markingOracle : ((Qubit[], Qubit) => Unit is Adj), register : Qubit[], iterations : Int) : Unit is Adj {
    // We convert the marking oracle in a phase flip oracle
    let phaseOracle = oracleConverter(markingOracle, _);
    // We put the array in a equal superposition
    ApplyToEachA(H, register);
    // We apply Uω and Us iterations
    for (i in 1..iterations) {
        // Uω
        phaseOracle(register);
        // Us
        ApplyToEachA(H, register);
        ApplyToEachA(X, register);
        (Controlled Z)(Most(register), Tail(register));
        ApplyToEachA(X, register);
        ApplyToEachA(H, register);
    }
}

In [ ]:
%kata T23_GroversAlgorithm_Test 

open Microsoft.Quantum.Measurement as Measurement;

operation GroversAlgorithm (V : Int, oracle : ((Qubit[], Qubit) => Unit is Adj)) : Int[] {
    mutable coloring = new Int[V];
    using ((register, output) = (Qubit[2 * V], Qubit())) {
        mutable correct = false;
        mutable iterations = 1;
        repeat {
            Message($"Trying iteration {iterations}");
            groverAlgorithm(oracle, register, iterations);
            let temp = Measurement.MultiM(register);
            oracle(register, output);
            if (Measurement.MResetZ(output) == One) {
                set correct = true;
                set coloring = MeasureColoring(V, register);
            }
            ResetAll(register);
        }
        until (correct or iterations > 10)
        fixup {
            set iterations += 1;
        }
        if (not correct) {
            fail "No valid coloring was found";
        }
    }
    return coloring;
}

[Return to task 2.3 of the Graph Coloring kata.](./GraphColoring.ipynb#Task-2.3.-Using-Grover's-search-to-find-vertex-coloring)